In [1]:
#Need to remove hardcoding but needed for now
%env LD_LIBRARY_PATH=/home/zach/anaconda3/envs/research/lib/
%env TF_GPU_ALLOCATOR=cuda_malloc_async

env: LD_LIBRARY_PATH=/home/zach/anaconda3/envs/research/lib/
env: TF_GPU_ALLOCATOR=cuda_malloc_async


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
from tensorflow import keras
import os
from official.nlp import optimization  # to create AdamW optimizer
import tensorflow_addons as tfa
f1 = tfa.metrics.F1Score(num_classes=1, average='macro')

tf.get_logger().setLevel('ERROR')



2022-05-24 23:05:53.884511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 23:05:53.888053: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 23:05:53.888373: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 23:05:53.888861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
print(tf.__version__)

2.9.0


In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
os.environ['LD_LIBRARY_PATH']

'/home/zach/anaconda3/envs/research/lib/'

## Data Import and Cleaning

In [6]:
train = pd.read_csv('data/hateval2019_en_train.csv')
test = pd.read_csv('data/hateval2019_en_test.csv')
val = pd.read_csv('data/hateval2019_en_dev.csv')

train = train.drop(['TR','AG'],1)
test = test.drop(['TR','AG'],1)
val = val.drop(['TR','AG'],1)

/tmp/ipykernel_244221/675437588.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train = train.drop(['TR','AG'],1)
/tmp/ipykernel_244221/675437588.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test = test.drop(['TR','AG'],1)
/tmp/ipykernel_244221/675437588.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  val = val.drop(['TR','AG'],1)


In [7]:
train.head()


,id,text,HS
0,201,"Hurray, saving us $$$ in so many ways @potus @...",1
1,202,Why would young fighting age men be the vast m...,1
2,203,@KamalaHarris Illegals Dump their Kids at the ...,1
3,204,NY Times: 'Nearly All White' States Pose 'an A...,0
4,205,Orban in Brussels: European leaders are ignori...,0


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.regularizers import L1,L2, l1_l2
import io

In [9]:
def normalize_tweet(text):
    """
    Removes hashtags, @s, links, and punctuation
    :param text:Text to be cleaned
    :return: text with mentions, hashtages, and urls removes
    """
    processed_text = text.lower()
    processed_text = re.sub(r"(?:\@|http?\://|https?\://|www|t\.)\S+", "", processed_text)
    processed_text = re.sub(r"(?:\.|,|\?|-)", " ", processed_text)
    processed_text = re.sub(r"(?:\@|http?\://|https?\://|www|\.com)", "", processed_text)
    processed_text = re.sub(r'[^\w\s]', '', processed_text)
    processed_text = " ".join(processed_text.split())
    return processed_text

def x_y_split(data):
    """splits and X and y from dataframe

    Args:
        data:dataframe to split from

    Returns:
        tuple:X data, y data
    """
    X = data['text']
    X = X.apply(normalize_tweet)
    y = data['HS']
    return X, y

## Split sequences into train, validation, and test sets

In [10]:
#Split x and ys
x_train, y_train = x_y_split(train)
print(x_train)
#x_train.to_csv('data/x_train.csv')
#y_train.to_csv('data/y_train.csv')

0       hurray saving us in so many ways lockthemup bu...
1       why would young fighting age men be the vast m...
2       illegals dump their kids at the border like ro...
3       ny times nearly all white states pose an array...
4       orban in brussels european leaders are ignorin...
                              ...                        
8995                  i am proud to be a hysterical woman
8996    hollywood is complicit in the rape and sexual ...
8997    what a fucking cunt i hate seeing kids getting...
8998                                hysterical woman like
8999    nearly every woman i know has metoo in their f...
Name: text, Length: 9000, dtype: object


In [11]:
#Split x and ys
x_test, y_test = x_y_split(test)
print(x_test)
#x_test.to_csv('data/x_test.csv')
#y_test.to_csv('data/y_test.csv')

0       oh i could have gone on about taxes since the ...
1       several of the wild fires in california and co...
2       my question is how do you resettle a refugee a...
3       europe youve got a problem we must hurry and b...
4       this is outrageous stopillegalimmigration meri...
                              ...                        
2995    you can never take a l off a real bitch im hot...
2996    likes to call me a cunt a bitch but i tell him...
2997    1 never said you were taught 2 you called me b...
2998    if i see and know you a hoe why would i hit yo...
2999     you be chasing them hoes fuck what a bitch think
Name: text, Length: 3000, dtype: object


In [12]:
#Split x and ys
x_val, y_val = x_y_split(val)
print(x_val)
#x_val.to_csv('data/x_val.csv')
#y_val.to_csv('data/y_val.csv')

0      i swear im getting to places just in the nick ...
1      im an immigrant and trump is right on immigrat...
2      illegalimmigrants illegalaliens electoralsyste...
3      we have our own invasion issues with mexicans ...
4      worker charged with sexually molesting eight c...
                             ...                        
995                     you unfollowed me fuck you pussy
996    stfu bitch and you go make some satanic music ...
997    honey as a fellow white chick let me tell you ...
998    i hate bitches who talk about niggaz with kids...
999    you won the life time recipient for hysterical...
Name: text, Length: 1000, dtype: object


In [13]:
import tensorflow_hub as hub
import tensorflow_text as text  # A dependency of the preprocessing model

In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [15]:
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [16]:
def build_classifier_model(do,trainable=False):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=trainable, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(do)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)
    return tf.keras.Model(text_input, net)




In [17]:

loss = tf.keras.losses.BinaryCrossentropy()
learning_rates = [5e-5, 1e-4, 3e-5, 3e-5]
learning_rates_text = ['5e-5', '1e-4', '3e-5', '3e-5']
batch_sizes = [8, 16]
dropout = [.5,.75,.25,0.0,.1]
results = {}
results_acc = {}
for do in dropout:
    for bs in batch_sizes:
        for lr in range(len(learning_rates)):
            string = 'lr{lr}_bs{bs}_do{do}'.format(lr=learning_rates_text[lr],bs=bs,do=do)
            #filepath = 'checkpoints/bert_finetuned/lr{lr}_bs{bs}.hdf5'.format(lr=learning_rates_text[lr],bs=bs)
            #checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 #    monitor='val_f1_score',
                                                  #  save_best_only=True,
                                                   #  verbose=1,)
            #callbacks = [checkpoint]

            bert = build_classifier_model(do,trainable=True)
            bert.compile(loss = loss, optimizer = Adam(learning_rate=learning_rates[lr]), metrics = [f1,'accuracy'])
            history = bert.fit(x=x_train,
                                y=y_train,batch_size=bs,
                                validation_data=(x_val,y_val),
                                epochs=4)

            max_f1 = max(history.history['val_f1_score'])
            max_acc = max(history.history['val_accuracy'])
            results[string] = max_f1
            results_acc[string] = max_acc

Epoch 1/4
1125/1125 [==============================] - 192s 164ms/step - loss: 0.5050 - f1_score: 0.5919 - accuracy: 0.7496 - val_loss: 0.5464 - val_f1_score: 0.5985 - val_accuracy: 0.7280
Epoch 2/4
1125/1125 [==============================] - 187s 166ms/step - loss: 0.3462 - f1_score: 0.5919 - accuracy: 0.8514 - val_loss: 0.5355 - val_f1_score: 0.5985 - val_accuracy: 0.7480
Epoch 3/4
1125/1125 [==============================] - 189s 168ms/step - loss: 0.1949 - f1_score: 0.5919 - accuracy: 0.9267 - val_loss: 0.7836 - val_f1_score: 0.5985 - val_accuracy: 0.7690
Epoch 4/4
1125/1125 [==============================] - 187s 166ms/step - loss: 0.0988 - f1_score: 0.5919 - accuracy: 0.9651 - val_loss: 0.9103 - val_f1_score: 0.5985 - val_accuracy: 0.7410
Epoch 1/4
1125/1125 [==============================] - 192s 165ms/step - loss: 0.6505 - f1_score: 0.5925 - accuracy: 0.6423 - val_loss: 0.6838 - val_f1_score: 0.5985 - val_accuracy: 0.5730
Epoch 2/4
1125/1125 [==============================] - 

In [18]:
results

{'lr5e-5_bs8_do0.5': 0.5984582901000977,
 'lr1e-4_bs8_do0.5': 0.5984582901000977,
 'lr3e-5_bs8_do0.5': 0.5984582901000977,
 'lr5e-5_bs16_do0.5': 0.5984582901000977,
 'lr1e-4_bs16_do0.5': 0.5984582901000977,
 'lr3e-5_bs16_do0.5': 0.5984582901000977,
 'lr5e-5_bs8_do0.75': 0.5984582901000977,
 'lr1e-4_bs8_do0.75': 0.5984582901000977,
 'lr3e-5_bs8_do0.75': 0.5984582901000977,
 'lr5e-5_bs16_do0.75': 0.5984582901000977,
 'lr1e-4_bs16_do0.75': 0.5984582901000977,
 'lr3e-5_bs16_do0.75': 0.5984582901000977,
 'lr5e-5_bs8_do0.25': 0.5984582901000977,
 'lr1e-4_bs8_do0.25': 0.5984582901000977,
 'lr3e-5_bs8_do0.25': 0.5984582901000977,
 'lr5e-5_bs16_do0.25': 0.5984582901000977,
 'lr1e-4_bs16_do0.25': 0.5984582901000977,
 'lr3e-5_bs16_do0.25': 0.5984582901000977,
 'lr5e-5_bs8_do0.0': 0.5984582901000977,
 'lr1e-4_bs8_do0.0': 0.5984582901000977,
 'lr3e-5_bs8_do0.0': 0.5984582901000977,
 'lr5e-5_bs16_do0.0': 0.5984582901000977,
 'lr1e-4_bs16_do0.0': 0.5984582901000977,
 'lr3e-5_bs16_do0.0': 0.5984582901

In [19]:
results2 = {}
results2_acc = {}

In [24]:
for do in dropout:
    for bs in batch_sizes:
        for lr in range(len(learning_rates)):
            string = 'lr{lr}_bs{bs}_do{do}'.format(lr=learning_rates_text[lr],bs=bs,do=do)
            #filepath = 'checkpoints/bert_finetuned/lr{lr}_bs{bs}.hdf5'.format(lr=learning_rates_text[lr],bs=bs)
            #checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 #    monitor='val_f1_score',
                                                  #  save_best_only=True,
                                                   #  verbose=1,)
            #callbacks = [checkpoint]

            bert = build_classifier_model(do,trainable=False)
            bert.compile(loss = loss, optimizer = Adam(learning_rate=learning_rates[lr]), metrics = [f1,'accuracy'])
            history = bert.fit(x=x_train,
                                y=y_train,batch_size=bs,
                                validation_data=(x_val,y_val),
                                epochs=4)

            max_f1 = max(history.history['val_f1_score'])
            max_acc = max(history.history['val_accuracy'])
            results2[string] = max_f1
            results2_acc[string] = max_acc

Epoch 1/4
1125/1125 [==============================] - 64s 54ms/step - loss: 0.7664 - f1_score: 0.5925 - accuracy: 0.5293 - val_loss: 0.6778 - val_f1_score: 0.5985 - val_accuracy: 0.5690
Epoch 2/4
1125/1125 [==============================] - 61s 54ms/step - loss: 0.7499 - f1_score: 0.5919 - accuracy: 0.5382 - val_loss: 0.6674 - val_f1_score: 0.5985 - val_accuracy: 0.5900
Epoch 3/4
1125/1125 [==============================] - 63s 56ms/step - loss: 0.7402 - f1_score: 0.5919 - accuracy: 0.5396 - val_loss: 0.6601 - val_f1_score: 0.5985 - val_accuracy: 0.5920
Epoch 4/4
1125/1125 [==============================] - 60s 54ms/step - loss: 0.7188 - f1_score: 0.5919 - accuracy: 0.5608 - val_loss: 0.6564 - val_f1_score: 0.5985 - val_accuracy: 0.5850
Epoch 1/4
1125/1125 [==============================] - 65s 53ms/step - loss: 0.7571 - f1_score: 0.5925 - accuracy: 0.5282 - val_loss: 0.6776 - val_f1_score: 0.5985 - val_accuracy: 0.5770
Epoch 2/4
1125/1125 [==============================] - 61s 54ms/s

In [21]:
results2

{'lr5e-5_bs8_do0.5': 0.5984582901000977,
 'lr1e-4_bs8_do0.5': 0.5984582901000977,
 'lr3e-5_bs8_do0.5': 0.5984582901000977,
 'lr5e-5_bs16_do0.5': 0.5984582901000977,
 'lr1e-4_bs16_do0.5': 0.5984582901000977,
 'lr3e-5_bs16_do0.5': 0.5984582901000977,
 'lr5e-5_bs8_do0.75': 0.5984582901000977,
 'lr1e-4_bs8_do0.75': 0.5984582901000977,
 'lr3e-5_bs8_do0.75': 0.5984582901000977,
 'lr5e-5_bs16_do0.75': 0.5984582901000977,
 'lr1e-4_bs16_do0.75': 0.5984582901000977,
 'lr3e-5_bs16_do0.75': 0.5984582901000977,
 'lr5e-5_bs8_do0.25': 0.5984582901000977,
 'lr1e-4_bs8_do0.25': 0.5984582901000977,
 'lr3e-5_bs8_do0.25': 0.5984582901000977,
 'lr5e-5_bs16_do0.25': 0.5984582901000977,
 'lr1e-4_bs16_do0.25': 0.5984582901000977,
 'lr3e-5_bs16_do0.25': 0.5984582901000977,
 'lr5e-5_bs8_do0.0': 0.5984582901000977,
 'lr1e-4_bs8_do0.0': 0.5984582901000977,
 'lr3e-5_bs8_do0.0': 0.5984582901000977,
 'lr5e-5_bs16_do0.0': 0.5984582901000977,
 'lr1e-4_bs16_do0.0': 0.5984582901000977,
 'lr3e-5_bs16_do0.0': 0.5984582901

In [22]:
results_acc


{'lr5e-5_bs8_do0.5': 0.7689999938011169,
 'lr1e-4_bs8_do0.5': 0.5730000138282776,
 'lr3e-5_bs8_do0.5': 0.7910000085830688,
 'lr5e-5_bs16_do0.5': 0.7590000033378601,
 'lr1e-4_bs16_do0.5': 0.746999979019165,
 'lr3e-5_bs16_do0.5': 0.7680000066757202,
 'lr5e-5_bs8_do0.75': 0.7730000019073486,
 'lr1e-4_bs8_do0.75': 0.5730000138282776,
 'lr3e-5_bs8_do0.75': 0.7850000262260437,
 'lr5e-5_bs16_do0.75': 0.7630000114440918,
 'lr1e-4_bs16_do0.75': 0.5730000138282776,
 'lr3e-5_bs16_do0.75': 0.7720000147819519,
 'lr5e-5_bs8_do0.25': 0.5730000138282776,
 'lr1e-4_bs8_do0.25': 0.5730000138282776,
 'lr3e-5_bs8_do0.25': 0.7730000019073486,
 'lr5e-5_bs16_do0.25': 0.7639999985694885,
 'lr1e-4_bs16_do0.25': 0.7440000176429749,
 'lr3e-5_bs16_do0.25': 0.7760000228881836,
 'lr5e-5_bs8_do0.0': 0.7799999713897705,
 'lr1e-4_bs8_do0.0': 0.5730000138282776,
 'lr3e-5_bs8_do0.0': 0.7699999809265137,
 'lr5e-5_bs16_do0.0': 0.7699999809265137,
 'lr1e-4_bs16_do0.0': 0.7540000081062317,
 'lr3e-5_bs16_do0.0': 0.78299999237

In [25]:
results2_acc

{'lr5e-5_bs8_do0.5': 0.5920000076293945,
 'lr1e-4_bs8_do0.5': 0.6190000176429749,
 'lr3e-5_bs8_do0.5': 0.5839999914169312,
 'lr5e-5_bs16_do0.5': 0.5709999799728394,
 'lr1e-4_bs16_do0.5': 0.6119999885559082,
 'lr3e-5_bs16_do0.5': 0.5830000042915344,
 'lr5e-5_bs8_do0.75': 0.5770000219345093,
 'lr1e-4_bs8_do0.75': 0.609000027179718,
 'lr3e-5_bs8_do0.75': 0.5849999785423279,
 'lr5e-5_bs16_do0.75': 0.5770000219345093,
 'lr1e-4_bs16_do0.75': 0.5799999833106995,
 'lr3e-5_bs16_do0.75': 0.5799999833106995,
 'lr5e-5_bs8_do0.25': 0.6039999723434448,
 'lr1e-4_bs8_do0.25': 0.6340000033378601,
 'lr3e-5_bs8_do0.25': 0.597000002861023,
 'lr5e-5_bs16_do0.25': 0.5649999976158142,
 'lr1e-4_bs16_do0.25': 0.6069999933242798,
 'lr3e-5_bs16_do0.25': 0.5759999752044678,
 'lr5e-5_bs8_do0.0': 0.6029999852180481,
 'lr1e-4_bs8_do0.0': 0.6489999890327454,
 'lr3e-5_bs8_do0.0': 0.6060000061988831,
 'lr5e-5_bs16_do0.0': 0.6159999966621399,
 'lr1e-4_bs16_do0.0': 0.597000002861023,
 'lr3e-5_bs16_do0.0': 0.5740000009536

In [26]:
from sklearn.metrics import classification_report
#model = tf.keras.models.load_model('checkpoints/fast_text_model.hdf5')
#print(val)
pred = bert.predict(x_val)
pred = np.round(pred)
#print(pred)
print(classification_report(y_val,pred))

32/32 [==============================] - 6s 185ms/step
              precision    recall  f1-score   support

           0       0.60      0.91      0.73       573
           1       0.63      0.19      0.29       427

    accuracy                           0.61      1000
   macro avg       0.61      0.55      0.51      1000
weighted avg       0.61      0.61      0.54      1000

